In [35]:
import getpass
import os
from dotenv import load_dotenv 
import pandas as pd
from langchain_openai import OpenAIEmbeddings

In [36]:
load_dotenv()

True

In [37]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [38]:
df = pd.read_csv("dataset/diseases_symptoms.tsv", sep="\t")

df.head()

,disease,symptoms
0,Postoperative Complications,"Abdomen, Abdominal Pain, Abnormal, Acquired, A..."
1,Brain Diseases,"Abdomen, Abdominal Pain, Abnormal, Acquired, A..."
2,Cerebral Infarction,"Abdominal Pain, Abnormal, Acquired, Acute Coro..."
3,Pain,"Abdomen, Abdominal Pain, Abnormal, Acute, Acut..."
4,Epilepsy,"Abdomen, Abdominal Pain, Abnormal, Acute, Agno..."


In [39]:
symptoms_each_disease = df["symptoms"].tolist()

In [40]:
import re

def clean_symptoms(symptoms_list):
  cleaned_symptoms = []
  for symptoms in symptoms_list:
      if isinstance(symptoms, list):
          symptoms = ','.join(symptoms)
      cleaned = re.sub(r',+', ',', symptoms).strip(', ').strip()
      if (len(cleaned) == 0):
        print("true")
      cleaned_symptoms.append(cleaned)

  return cleaned_symptoms

def seperate_symptoms(symptoms_each_disease):
  return [symptoms.split(', ') for symptoms in symptoms_each_disease]

cleaned_symptoms_each_disease = clean_symptoms(symptoms_each_disease)

seperated_symptoms_each_disease = seperate_symptoms(cleaned_symptoms_each_disease)

In [41]:
seperated_symptoms_each_disease[-1]

['Coma']

In [42]:
main_df = pd.read_csv('dataset/Combined-Output.tsv', sep="\t")

main_df.head()

,Unnamed: 0,MeSH Symptom Term,MeSH Disease Term,PubMed occurrence,TFIDF score,MeSH Disease ID,MeSH Symptom ID
0,0,"Aging, Premature",Respiratory Syncytial Virus Infections,1,3.464551,D018357,D019588
1,1,"Aging, Premature",Orthomyxoviridae Infections,1,3.464551,D009976,D019588
2,2,"Aging, Premature",HIV Infections,3,10.393654,D015658,D019588
3,3,"Aging, Premature",Acquired Immunodeficiency Syndrome,3,10.393654,D000163,D019588
4,4,"Aging, Premature",Breast Neoplasms,1,3.464551,D001943,D019588


In [43]:
main_df = main_df[(main_df['MeSH Symptom Term'] != main_df['MeSH Disease Term']) & (main_df['PubMed occurrence'] > 32)]

main_df.head()

,Unnamed: 0,MeSH Symptom Term,MeSH Disease Term,PubMed occurrence,TFIDF score,MeSH Disease ID,MeSH Symptom ID
134,134,Fever,HIV Infections,153,94.697458,D015658,D005334
135,135,Fever,Acquired Immunodeficiency Syndrome,74,45.801385,D000163,D005334
136,136,Fever,Breast Neoplasms,107,66.226327,D001943,D005334
138,138,Fever,Skin Neoplasms,44,27.233256,D012878,D005334
146,146,Fever,"Arthritis, Rheumatoid",54,33.422632,D001172,D005334


In [142]:
proccessed_df = pd.read_csv('dataset/diseases_symptoms_processed.tsv', sep="\t")

proccessed_df.head()

,disease,symptoms
0,Postoperative Complications,Abdomen|Abdominal Pain|Abnormal|Acquired|Acute...
1,Brain Diseases,Abdomen|Abdominal Pain|Abnormal|Acquired|Acute...
2,Cerebral Infarction,Abdominal Pain|Abnormal|Acquired|Acute Coronar...
3,Pain,Abdomen|Abdominal Pain|Abnormal|Acute|Acute Co...
4,Epilepsy,Abdomen|Abdominal Pain|Abnormal|Acute|Agnosia|...


## LLMs for Medical Analysis

This section uses LangChain and OpenAI to analyze medical symptoms.


In [143]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document


In [144]:
proccessed_df['symptoms_list'] = proccessed_df['symptoms'].str.split('|')

proccessed_df.head()

,disease,symptoms,symptoms_list
0,Postoperative Complications,Abdomen|Abdominal Pain|Abnormal|Acquired|Acute...,"[Abdomen, Abdominal Pain, Abnormal, Acquired, ..."
1,Brain Diseases,Abdomen|Abdominal Pain|Abnormal|Acquired|Acute...,"[Abdomen, Abdominal Pain, Abnormal, Acquired, ..."
2,Cerebral Infarction,Abdominal Pain|Abnormal|Acquired|Acute Coronar...,"[Abdominal Pain, Abnormal, Acquired, Acute Cor..."
3,Pain,Abdomen|Abdominal Pain|Abnormal|Acute|Acute Co...,"[Abdomen, Abdominal Pain, Abnormal, Acute, Acu..."
4,Epilepsy,Abdomen|Abdominal Pain|Abnormal|Acute|Agnosia|...,"[Abdomen, Abdominal Pain, Abnormal, Acute, Agn..."


In [145]:
proccessed_df['content'] = proccessed_df.apply(lambda row: f"Disease: {row['disease']}. Symptoms: {', '.join(row['symptoms_list'])}", axis=1)

In [146]:
llm = ChatOpenAI(temperature=0, max_completion_tokens=1024, model="gpt-4.1")

In [147]:
langchain_docs = []
for idx, row in proccessed_df.iterrows():
    disease = row['disease']
    symptoms_text = ", ".join(row['symptoms_list']) if isinstance(row['symptoms_list'], list) else row['symptoms']
    content = f"Bệnh: {disease}. Triệu chứng: {symptoms_text}"
    
    # Tạo Document với metadata là dictionary chỉ chứa string
    doc = Document(
        page_content=content,
        metadata={"disease": disease, "symptoms_summary": symptoms_text[:500]}
    )
    langchain_docs.append(doc)

In [148]:
embeddings = OpenAIEmbeddings()

In [149]:
vectorstore = Chroma.from_documents(
    documents=langchain_docs,
    embedding=embeddings
)

In [150]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
import numpy as np
import xgboost as xgb

import torch

In [151]:
print(torch.cuda.is_available())

False


In [152]:
def training_xg_boost_model():
    # Tạo tập hợp tất cả triệu chứng
    all_symptoms = set()
    for symptom_list in proccessed_df['symptoms_list']:
        all_symptoms.update(symptom_list)

    # Tạo ma trận one-hot cho các triệu chứng
    X = pd.DataFrame(0, index=np.arange(len(proccessed_df)), columns=list(all_symptoms))
    for idx, row in proccessed_df.iterrows():
        for symptom in row['symptoms_list']:
            X.loc[idx, symptom] = 1
    
    # Mã hóa nhãn bệnh
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(proccessed_df['disease'])

    # sample_weights = compute_sample_weight(
    #     class_weight='balanced',
    #     y=proccessed_df['disease'] #provide your own target name
    # )
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.333, random_state=42)

    y_train = label_encoder.fit_transform(y_train)

    # fit_kwargs_native = {'objective': 'binary:logistic', 'tree_method': 'hist', 'device': 'gpu'}

    # dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=list(X_train.columns))

    # Train model with proper parameters
    model = xgb.XGBClassifier(
        n_estimators=100,
        random_state=42,
        use_label_encoder=False,  # Important for newer XGBoost versions
        eval_metric='mlogloss',  # Add evaluation metric
    )

    # model = xgb.train(dtrain=dtrain, params=fit_kwargs_native, num_boost_round=100)
    
    model.fit(X=X_train, y=y_train)

    # Make predictions on test data
    predictions = model.predict(X_test)

    # Print evaluation metrics
    print(f"Model Accuracy: {accuracy_score(y_test, predictions):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, predictions, zero_division=0))

    # Store all symptoms and mappings for later use
    model.all_symptoms = list(all_symptoms)
    model.label_encoder = label_encoder
    
    # Store the disease names in the correct order matching model.classes_
    model.disease_names = label_encoder.inverse_transform(np.arange(len(label_encoder.classes_)))

    return model

In [153]:
xgboost_model = training_xg_boost_model()

d:\WorkSpace\KHOA_LUAN_TOT_NGHIEP\healer-rag\.venv\lib\site-packages\xgboost\training.py:183: UserWarning: [22:39:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  [xgb.callback.LearningRateScheduler(custom_rates)]


Model Accuracy: 0.0008

Classification Report:
              precision    recall  f1-score   support

           0       0.00      1.00      0.00         1
           2       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          21       0.00      0.00      0.00         1
          24       0.00      0.00      0.00         1
          26       0.00      0.00      0.00         1
          28       0.00      0.00      0.00         1
          29       0.00      0.00      0.00         1
          33       0.00      0.00      0.00         1
          35       0.00      0.00      0.00         1
          36       0.00      0.00      0.00         1
          40       0.00      0.00      0.00         1
          43       0.00      0.00 

In [ ]:
def generate_follow_up_questions(chat_history, reported_symptoms):
    # Tạo prompt cho LLM
    symptoms_text = ", ".join(reported_symptoms)
    prompt = f"""
    Dựa trên các triệu chứng đã báo cáo: {symptoms_text}
    
    Hãy đưa ra một vài (từ 3 trở lên) câu hỏi quan trọng nhất để xác định chính xác bệnh.
    Mỗi câu hỏi nên hỏi về một triệu chứng cụ thể và có thể trả lời bằng có hoặc không, các câu hỏi này nên được đặt ra cho bệnh nhân để xác định chính xác bệnh nên cần phải hỏi khác nhau hoặc không lặp lại triệu chứng đã báo cáo.
    """
    
    # Sử dụng retriever để lấy thông tin liên quan
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever
    )
    
    # Thực hiện truy vấn
    result = qa({"question": prompt, "chat_history": chat_history})
    
    # Xử lý kết quả
    questions = result["answer"].strip().split("\n")
    questions = [q.strip() for q in questions if "?" in q]
    
    return questions[:3]

In [ ]:
def predict_disease(model, symptoms):
    # Tạo vector triệu chứng
    symptoms_vector = pd.DataFrame(0, index=[0], columns=model.all_symptoms)
    
    # Đánh dấu các triệu chứng có mặt
    for symptom in symptoms:
        if symptom in model.all_symptoms:
            symptoms_vector.loc[0, symptom] = 1

    print()
    
    # Dự đoán xác suất của các lớp
    probabilities = model.predict_proba(symptoms_vector)[0]
    
    # Lấy tên bệnh tương ứng với chỉ số lớp
    results = []
    for i, prob in enumerate(probabilities):
        disease_name = model.disease_names[i]
        results.append((disease_name, prob))
    
    # Sắp xếp theo xác suất giảm dần
    results.sort(key=lambda x: x[1], reverse=True)
    
    return results[:5]  # Trả về 5 bệnh có xác suất cao nhất

In [161]:
def main_diagnostic_flow():
    # Khởi tạo
    chat_history = []
    reported_symptoms = []
    
    # Nhận triệu chứng ban đầu
    initial_symptoms = input("Nhập các triệu chứng ban đầu, cách nhau bằng dấu phẩy: ")
    reported_symptoms = [s.strip() for s in initial_symptoms.split(",")]
    
    # Thực hiện 3 vòng hỏi
    for i in range(3):
        questions = generate_follow_up_questions(chat_history, reported_symptoms)
        
        print(f"\nVòng hỏi {i+1}:")
        for question in questions:
            print(question)
            answer = input("Trả lời (có/không): ").strip().lower()
            
            # Cập nhật lịch sử đối thoại
            chat_history.append((question, answer))
            
            # Thêm triệu chứng nếu câu trả lời là có
            if answer == "có":
                symptom = question.replace("Bạn có bị ", "").replace(" không?", "")
                reported_symptoms.append(symptom)

    print("\nTriệu chứng đã báo cáo: ", reported_symptoms)
    
    # Dự đoán bệnh
    predictions = predict_disease(model=xgboost_model, symptoms=reported_symptoms)
    
    # Hiển thị kết quả
    print("\nDự đoán bệnh dựa trên triệu chứng của bạn:")
    for disease, probability in predictions:
        print(f"{disease}: {probability*100:.2f}%")

In [162]:
main_diagnostic_flow()


Vòng hỏi 1:
1. Bạn có bị nôn mửa không?
2. Bạn có bị tiêu chảy không?
3. Mắt hoặc da của bạn có bị vàng (vàng da, vàng mắt) không?

Vòng hỏi 2:
1. Gần đây bạn có thấy mắt hoặc da mình bị vàng (vàng da, vàng mắt) không? (Có/Không)
2. Bạn có bị sốt cao liên tục không? (Có/Không)
3. Bạn có bị đau bụng kèm theo nôn mửa và tiêu chảy không? (Có/Không)

Vòng hỏi 3:
1. Bạn có bị vàng da hoặc vàng mắt không? (Có/Không)
2. Bạn có bị sốt cao liên tục không? (Có/Không)
3. Bạn có bị đau bụng kèm theo nôn mửa và tiêu chảy không? (Có/Không)

Triệu chứng đã báo cáo:  ['Headache', 'high fever', 'cough', 'stomache', 'muscle pain', 'yellow eyes', '1. nôn mửa', '2. tiêu chảy', '3. Mắt hoặc da của bạn có bị vàng (vàng da, vàng mắt)', '2. sốt cao liên tục (Có/Không)', '3. đau bụng kèm theo nôn mửa và tiêu chảy (Có/Không)', '1. vàng da hoặc vàng mắt (Có/Không)', '2. sốt cao liên tục (Có/Không)', '3. đau bụng kèm theo nôn mửa và tiêu chảy (Có/Không)']

Dự đoán bệnh dựa trên triệu chứng của bạn:
1: 0.04%
3: 0